# CoronaVirus Covid19 India Lockdown 2.0

In [150]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
import networkx as nx

# Karnataka Coronavirus Data

In [143]:
#url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Karnataka'
#all_tables = pd.read_html(url)

# we  need 6 th table that is all_tables[5]
#rawCases = all_tables[5]
rawCases = pd.read_csv("https://raw.githubusercontent.com/muthukumar-ram/covid19indialive/master/dataset/KARawCases.csv")

In [144]:
genderGrp=rawCases.groupby(by='Gender').size().reset_index(name='count')
genderGrp=genderGrp.rename(columns={'count':'Confirmed'})


px.bar(
    genderGrp,
    x = "Gender",
    y = "Confirmed",
    title= "Confirmed Cases based on Gender", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=500
)

In [145]:

#ageGroup=rawCases.groupby(by='AgeRange')
#ageGroup.AgeRange.count().plot(kind='bar');

ageGroup=rawCases.groupby(by='AgeRange').size().reset_index(name='count')
ageGroup=ageGroup.rename(columns={'AgeRange':'Age Group','count':'Confirmed'})


px.bar(
    ageGroup,
    x = "Age Group",
    y = "Confirmed",
    title= "Confirmed Cases based on Age of Patients", # the axis names
    color_discrete_sequence=["purple"], 
    height=500,
    width=800
)


In [146]:
districtData=rawCases.groupby(by='Location').size().reset_index(name='count')

In [147]:
sortedDistrict=districtData.sort_values(by='count',ascending=False)
sortedDistrict=sortedDistrict.rename(columns={'Location':'Districts','count':'Confirmed'})

In [148]:
fig = px.scatter(sortedDistrict.head(), x="Districts", y="Confirmed", size="Confirmed", color="Districts",
               hover_name="Districts", size_max=60)
fig.update_layout(
    title=str(5) +" Worst hit districts in Karnataka",
    xaxis_title="Districts",
    yaxis_title="Confirmed Cases",
    width = 700
    )
fig.show();

In [149]:
# Preparing nodes and edges from dataframe
originLists = []
nodes = []
for i,rows in rawCases['Origin'].iteritems():
    if '#' in rows:
        no = rows.split('#')[1].split(' ')[0]
        node = str(rawCases['Patient Number'][i])
        originLists.append((no,node))
        nodes.append(node)
    else:
        node = str(rawCases['Patient Number'][i])
        nodes.append(node)

In [151]:
# creating graph object and adding nodes and edges to the network
G = nx.Graph()
for i in nodes:
    G.add_node(i)
G.add_edges_from(originLists)
# positioning the nodes
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.node[n]['pos'] = p


In [152]:
# getting ready for plotting
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])


In [153]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = adjacencies[0] +' # of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Karnataka Person to Person Network Tracing',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="No. of contacts made",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

In [154]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
iplot(fig)